# Selects concrete beams with over a certain amount of reinforcement in percentage

In [5]:
import pandas as pd
import numpy as np
import os
import sys
import comtypes.client
import lib.csiapi

import json
import tkinter as tk
from tkinter import filedialog
import lib.csiapi as csiapi

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Initialization of ETABSObject
ETABS_path_exe = r'C:/Program Files/Computers and Structures/ETABS 21_2/ETABS.exe'
if not os.path.isfile(ETABS_path_exe):
    root = tk.Tk()
    root.withdraw()
    ETABS_path = filedialog.askopenfilename(parent=root)
    root.destroy()
# create API helper object
helper = comtypes.client.CreateObject("ETABSv1.Helper")
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
print("Succesfully created helper object")

try:
    #get the active ETABS object
    ETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    SapModel = ETABSObject.SapModel
    print("Succesfully created SapModel")
except (OSError, comtypes.COMError):  
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

# create folder for temporary csv files
csv_folder_path = os.getcwd() + r'\csv'
if not os.path.exists(csv_folder_path):
    os.makedirs(csv_folder_path)


Succesfully created helper object
Succesfully created SapModel


In [26]:
SapModel.Analyze.RunAnalysis()
SapModel.DesignConcrete.StartDesign()

0

In [119]:
as_percent_threshold = 1.2

table_key = 'Frame Section Property Definitions - Summary'
table = csiapi.get_table_display(SapModel,table_key)
table_simple = table[['Name','Material','Area']]
table_key = 'Concrete Beam Design Summary - ACI 318-19'
table2 = csiapi.get_table_display(SapModel,table_key)
table2_simple = table2[['Story','Label','UniqueName','DesignSect','Station','AsTopTotal','AsBotTotal','AtTrnTotal']]

newtable = table2_simple.merge(table_simple, left_on = 'DesignSect', right_on = 'Name')

newtable['AsTopPercentage'] = newtable['AsTopTotal'].div(newtable['Area'])*100
newtable['AsBotPercentage'] = newtable['AsBotTotal'].div(newtable['Area'])*100

as_exceeds_threshold_table = newtable[(newtable['AsTopPercentage'] > as_percent_threshold) | (newtable['AsBotPercentage'] > as_percent_threshold)]

uniquenames = pd.Series(as_exceeds_threshold_table['UniqueName']).unique()

# uniquenamelist = uniquenames.to_numpy()
for i in uniquenames:
    ret = SapModel.FrameObj.SetSelected(str(i.item()),True)